<font color="blue"> **This notebook cleans up the data read from text files and saves them to two tables (image_stats, image_address) in the 'coffee_detail' sql database.** </font>

In [20]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import json
import re

In [21]:
dbname = 'coffee_detai1'
username = 'emclinden'
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

postgres://emclinden@localhost/coffee_detai1
True


<font color="blue"> **Eventually want to cat page_1 - page_x for full table** </font></bf>

In [22]:
with open('pages_1_19.txt', 'rb') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

#make the individual json object into a single json object
data_json_str = "[" + ','.join(data) + "]"

In [23]:
data = json.loads(data_json_str)
names = ['picture_id','user_id','static_file','date_posted','geo_loc','n_favs','n_views','n_comments','tag_raw','tag_text']

#create the dataframe 
mydf = pd.DataFrame(data,columns=names)

In [24]:
#do some cleaning to separate out lat and longitude and remove extra characters in user_id
ulist = []
latt = []
longt = []

for user in mydf.user_id:
    u = user.replace('<Flickr User ','')
    u = u.replace('>','')
    ulist.append(u)
    
for geo in mydf.geo_loc:
    l = geo[0]
    o = geo[1]
    latt.append(l); longt.append(o);
    
mydf.user_id = ulist
mydf = mydf.drop('geo_loc', 1)
mydf['latitude'] = latt
mydf['longitude'] = longt

In [25]:
#send cleaned up data frame to sql table
mydf.to_sql('image_stats', engine, if_exists='replace')

In [26]:
#now make a second df that will house picture location
#and will be stored in a second sql table
mydf2 = pd.DataFrame()
mydf2['picture_id'] = mydf.picture_id
mydf2['picture_address'] = mydf.static_file

In [27]:
#get rid of leading part of static address
mydf2['picture_address'].replace(regex=True,inplace=True,to_replace=r'https://farm\d\.staticflickr\.com/\d+/',value=r'')

newadd = []
for p in mydf2.picture_address:
    p = '/Users/emclinden/coffee_100115_011816/'+p
    newadd.append(p)

mydf2 = mydf2.drop('picture_address', 1)
mydf2['picture_address'] = newadd

In [28]:
#load second table containing image locations into sql db
mydf2.to_sql('image_address', engine, if_exists='replace')